# PyArrow

`PyArrow` - фреймворк, которая определяет стандартизированный и независящий от языка и архитектуры формат хранения табличных данных в памяти.

![](https://arrow.apache.org/img/copy.png)

![](https://arrow.apache.org/img/shared.png)

Базовая структура - массив

In [1]:
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds


ar = pa.array([1, 2, 3], pa.int64())

Массивы напоминают массивы `NumPy`, но у них нет поддержки большого количества функций для обработки и они не изменяемы. При этом можно производить вычисления:

In [2]:
pc.max(ar)

<pyarrow.Int64Scalar: 3>

и преобразовывать в `NumPy`

In [3]:
ar.to_numpy()

array([1, 2, 3])

Поддерживаются более сложные структуры данных:

In [4]:
fields = [
    ('name', pa.string()),
    ('age', pa.int32()),
]

struct_type = pa.struct(fields)
ar = pa.array([("Alex", 20), ("Andrew", 30)], type=struct_type)
ar

-- is_valid: all not null
-- child 0 type: string
  [
    "Alex",
    "Andrew"
  ]
-- child 1 type: int32
  [
    20,
    30
  ]

Таблицы - почти как `DataFrame`, набор именованных массивов.

In [5]:
ages = pa.array([20, 23, 25, 27, 31], type=pa.int8())
first_names = pa.array(["Alex", "Andrew", "Max", "Nick", "Peter"])

persons = pa.table([first_names, ages], names=["first_name", "age"])
persons


pyarrow.Table
first_name: string
age: int8
----
first_name: [["Alex","Andrew","Max","Nick","Peter"]]
age: [[20,23,25,27,31]]

Таблицы хранятся по столбцам

![](https://arrow.apache.org/img/simd.png)

К столбцам можно применять функции. Создадим большую таблицу и сравним производительность некоторых функций в `pandas` и `pyarrow`

In [6]:
import random
import re
import pandas as pd

import pyarrow as pa
import pyarrow.compute as pc


words = [word for line in open("../data/texts.txt", "r") for word in re.findall(r"\w+", line)]
entries = [
     [ 
        word, 
        random.random() * 10, 
        random.random() * 10, 
        random.randint(1, 10)
     ] 
 for word in words]

df = pd.DataFrame(entries, columns=["word", "f1", "f2", "f3"])
table = pa.Table.from_pandas(df)

Можно сравнить производительность похожих функций (например -- преобразование к верхнему регистру)

In [7]:
# 
%timeit pc.utf8_upper(table["word"])

%timeit df["word"].str.upper()

122 µs ± 6.26 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
1.03 ms ± 23.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


`PyArrow` предоставляет структуру, которая называется `Datset` для работы с большими данными, которая позволяет обрабатывать данные, разделяя их на более мелкие части.

In [8]:
# нативная поддержка S3 
# ds = ds.dataset("s3://my/a.parquet")

words_ds = ds.dataset(table)

for chunk in words_ds.to_batches(batch_size=2):
    print(chunk["word"])
    print(chunk.num_rows)

[
  "0",
  "Школа"
]
2
[
  "злословия",
  "учит"
]
2
[
  "прикусить",
  "язык"
]
2
[
  "Сохранится",
  "ли"
]
2
[
  "градус",
  "дискуссии"
]
2
[
  "в",
  "новом"
]
2
[
  "сезоне",
  "Великолепная"
]
2
[
  "Школа",
  "злословия"
]
2
[
  "вернулась",
  "в"
]
2
[
  "эфир",
  "после"
]
2
[
  "летних",
  "каникул"
]
2
[
  "в",
  "новом"
]
2
[
  "формате",
  "В"
]
2
[
  "истории",
  "программы"
]
2
[
  "это",
  "уже"
]
2
[
  "не",
  "первый"
]
2
[
  "ребрендинг",
  "Сейчас"
]
2
[
  "с",
  "трудом"
]
2
[
  "можно",
  "припомнить"
]
2
[
  "что",
  "начиналась"
]
2
[
  "Школа",
  "на"
]
2
[
  "канале",
  "Культура"
]
2
[
  "как",
  "стандартное"
]
2
[
  "ток",
  "шоу"
]
2
[
  "которое",
  "отличалось"
]
2
[
  "от",
  "других"
]
2
[
  "кухонными",
  "обсуждениями"
]
2
[
  "гостя",
  "что"
]
2
[
  "называется",
  "за"
]
2
[
  "глаза",
  "и"
]
2
[
  "неожиданными",
  "персонами"
]
2
[
  "в",
  "качестве"
]
2
[
  "ведущих",
  "Писательница"
]
2
[
  "Татьяна",
  "Толстая"
]
2
[
  "и",
  "сценаристк

# Встраиваемые базы данных

Использование встраиваемых баз данных позволяет в некоторых случаях упростить работу с большими данными локально. В `Pandas` встроены [функции](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html), которые с помощью [SQLAlchemy](https://www.sqlalchemy.org/) позволяют читать/писать данные из различных СУБД, в том числе [SQLite](https://sqlite.org) и [DuckDB](https://duckdb.org) 

In [9]:
from sqlalchemy import create_engine, text


df = pd.DataFrame([("Pavel", 20), ("Mark", 45)], columns=["name", "age"])

engine = create_engine("sqlite:///") # - будет создана БД в памяти текущего процесса 

# engine = create_engine("sqlite:///data.db")  # - содержимое БД будет записываться в файл

with engine.begin() as conn:
    conn.execute(
        text("""CREATE TABLE IF NOT EXISTS person (
                    name varchar(30),
                    age int
                );"""
        )
    )

    conn.execute(
        text("INSERT INTO person (name, age) VALUES ('Andrew', 25), ('Max', 30)")
    )

with engine.begin() as conn:
    df.to_sql("person", conn, if_exists="append", index=None)

    for l in conn.execute(text("SELECT * FROM person")):
        print(l)

df

('Andrew', 25)
('Max', 30)
('Pavel', 20)
('Mark', 45)


,name,age
0,Pavel,20
1,Mark,45


`DuckDB` хранит данные по столбцам и оптимизирована для `OLAP`

In [10]:
from sqlalchemy import create_engine, text

engine = create_engine("duckdb:///") # - будет создана БД в памяти текущего процесса 

# engine = create_engine("duckdb:///data.db")  # - содержимое БД будет записываться в файл

with engine.begin() as conn:
    conn.execute(
        text("""CREATE TABLE IF NOT EXISTS person (
                    name varchar(30),
                    age int
                );"""
        )
    )

    conn.execute(
        text("INSERT INTO person (name, age) VALUES ('Andrew', 25), ('Max', 30)")
    )

    for l in conn.execute(text("SELECT * FROM person")):
        print(l)

('Andrew', 25)
('Max', 30)


`DuckDB` позволяет работать с датафреймами `pandas` из текущего контекста, интерпретируя их в качестве таблиц

In [12]:
import duckdb

duckdb.query("SELECT name, age+10 FROM df WHERE name like '%%Pavel%%'")

┌─────────┬────────────┐
│  name   │ (age + 10) │
│ varchar │   int64    │
├─────────┼────────────┤
│ Pavel   │         30 │
└─────────┴────────────┘

Можно преобразовать результат обратно в датафрейм

In [13]:
duckdb.query("SELECT name, (age+10) as age FROM df").df()

,name,age
0,Pavel,30
1,Mark,55
